In [ ]:
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.wfr import *

# get key from keypairs.json
my_key = get_key('koray_data')

# move other processed files to processed files field
def move_opc_to_pc(resp, move_title, con_key):
    opc = resp.get('other_processed_files')
    pc = resp.get('processed_files')
    # if processed_files field already has values, exit
    if pc:
        print('There are files in processed_files field, expected empty')
        return False
    # see if there are other_processed_files to move
    if opc:
        titles = [i['title'] for i in opc]
        if move_title in titles:
            print(resp['accession'], 'files will move')
            move_item = [i for i in opc if i['title'] == move_title]
            assert len(move_item) == 1
            assert move_item[0]['type'] == 'preliminary'
            new_pc = move_item[0]['files']
            new_opc = [i for i in opc if i['title'] != move_title]
            # Time to patch
            patch_data = {}
            add_on = ""
            #if there is anthing left in opc, patch it, if not delete field
            if new_opc:
                patch_data['other_processed_files'] = opc
            else:
                add_on = 'delete_fields=other_processed_files'
            # patch with processed files
            patch_data['processed_files'] = new_pc
            print(patch_data)
            ff_utils.patch_metadata(patch_data, resp['uuid'], key = con_key, add_on = add_on)
            # update status of pc to status of set or exp
            release_files(resp['uuid'], new_pc, con_key)
            return True
        else:
            return False

In [ ]:
set_url = '/search/?experiments_in_set.experiment_type=TrAC-loop' + \
          '&type=ExperimentSetReplicate&limit=all'
run_sets = ff_utils.search_metadata(set_url , key=my_key)
set_w_apf = 0
exp_w_apf = 0
counter = 0
move_title = 'HiC Processing Pipeline - Preliminary Files'

print(len(run_sets), 'experiment sets in scope')
for a_set in run_sets:
    a_set_raw = ff_utils.get_metadata(a_set['uuid'],key=my_key, add_on='frame=raw')
    counter += 1
    print(counter, a_set['accession'])
    exps = a_set_raw['experiments_in_set']
    res =  move_opc_to_pc(a_set_raw,move_title,my_key)
    if res:
        set_w_apf += 1
    else:
        continue   
    for exp in exps:
        exp_resp = ff_utils.get_metadata(exp, key=my_key, add_on='frame=raw')
        res_e =  move_opc_to_pc(exp_resp,move_title,my_key)
        if res_e:
            exp_w_apf += 1
    print()

print(set_w_apf)
print(exp_w_apf)